# 0 : Importation des librairies

Pour ce projet, j'ai besoin d'importer : 
- librairies classiques de manipulations de données (**_pandas, numpy, scipy, sklearn_**)
- fonctions du dossier **_functions_**
- librairies classiques d'affichage (**_seaborn, matplotlib_**)

In [ ]:
%load_ext autoreload
%autoreload 2

# Importation des librairies classiques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Importation des modèles d'imputations ML
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer  # nécessaire
from sklearn.impute import IterativeImputer

# Importation des fonctions du fichier functions/processing_functions.py
import os
import sys
sys.path.append(os.path.abspath(".."))

from functions.processing_functions import *

: 

# 0.bis : Importation des fichiers

Le challenge ayant déja pris soin de séparer le dataset en un dataset d'entraînement et un dataset de test, je vais simplement fusionner les datasets **_x_train_** et **_y_train_** via la colonne **_DELIVERY_START_** préalablement lue comme une date. Cela me permettra de travailler ce dataset comme un seul et unique dataset et plus facilement récuperer les relations entre variables explicatives et cible.

In [ ]:
x_train = pd.read_csv('../data/raw/X_train_Wwou3IE.csv', parse_dates = ['DELIVERY_START']) # 10605 rows x 10 columns
y_train = pd.read_csv('../data/raw/y_train_jJtXgMX.csv', parse_dates = ['DELIVERY_START']) # 10605 rows x 2 columns

x_test = pd.read_csv('../data/raw/X_test_GgyECq8.csv', parse_dates = ['DELIVERY_START']) # 4942 rows x 10 columns

# Passage 
for df in [x_train, y_train, x_test]:
    df['DELIVERY_START'] = (
        pd.to_datetime(df['DELIVERY_START'], utc=True) # Convertit 03:30:00+02:00 en 01:30:00+00:00 UTC
        .dt.tz_convert("Europe/Paris") # Convertit 01:30:00+00:00 UTC en 03:30:00+02:00 Fuseau Paris
        .dt.tz_localize(None) # Supprime la notion de Fuseau, résultat : 03:30:00
    )

# Jointure de x_train et y_train sur 'DELIVERY_START'
df_train = pd.merge(x_train, y_train, on = 'DELIVERY_START') # 10605 rows x 11 columns

print(df_train['DELIVERY_START'].isna().sum())

# 1.a : Atomisation des données

Le marché Spot est un marché détérminant un prix pour chaque heure de la journée, il est donc crucial d'isoler cette information. Aussi, ce marché doit à tout prix faire converger l'offre et la demande à tout instant, il faut donc être en mesure de prédire au plus proche la consommation en électricité. Cette consommation dépend de différentes tendances à différentes échelles. On consomme plus en hier qu'en été, le matin et le soir qu'en milieu d'après-midi ou encore les jours de la semaine plutot que le week-end. Ainsi, il semble pertinent d'extraire et d'encoder toutes ces informations à partir de la variable **_DELIVERY_START_**.

La première étape de processing de ce projet consiste en l'atomisation des informations contenues dans **_DELIVERY_START_** en les informations suivantes : 

- **_hour_** : variable entière de 0 à 23 
- **_is_peak_hour_** : variable binaire _(1 = peak_hour, 2 = off_peak)_
- **_is_weekend_** : variable binaire _(1 = week-end, 2 = week-day)_
- **_day_of_week_** : variable entière de 1 à 7
- **_month_** : variable entière de 1 à 12
- **_season_** : variable entière : 1 = 'winter' ; 2 = 'spring' ; 3 = 'summer' ; 4 = 'autumn' (ca permettra au modèle de tourner, on pourra mettre en place ce labeling)

Dans un contexte de prédiction des heures de pointes, je considère qu'il vaut mieux avoir trop d'heure de pointes que pas assez et risquer de manquer de prédire une tension sur le marché et ainsi perdre beaucoup d'argent voire être sanctionné d'une pénalité sur le marché Intraday.

Dans cette objectif d'exploiter au mieux les informations fournies par ce dataset et dans un contexte de marché de l'énergie, il semble pertinent d'ajouter quelques variables explicatives.

J'introduis donc **_dispatchable_capacity_**, représentant la capacité totale pilotable (charbon, gaz, nucléaire), utile pour évaluer la capacité de réponse à la demande, **_load_vs_dispatchable_** qui construit le ratio entre demande et capacité pilotable et permet d'indiquer les potentielles tensions sur le marché. 

J'ajoute aussi **_renewable_share_**, mesurant la part des énergies renouvelables, afin d’anticiper les effets du merit order, où une forte disponibilité renouvelable tend à faire baisser les prix.

In [ ]:
# On applique la fonction extract_time_features implémentées dans le fichier functions.processing
print(df_train['DELIVERY_START'].dtype)
df_train = extract_time_features(df_train)
df_train = add_energy_features(df_train)

print("df_train :", df_train.shape)
display(df_train.head())

Visuellement, il est clair qu'il y'a deux blocs de missing values à traiter. D'une part, toutes les données de consommation ont été perdue entre le 01/07/2022 et le 22/08/2022, ceci correspond donc à 1262 lignes inutilisables tant l'information **_load_forecast_** est cruciale. 

Aussi, il semble inconsidérer de tenter une imputation via un KNN ou une régression classique qui viendrait ajouter énormément de bruit pour nos modèles ultérieures, étant donné que ce sont des données temporelles qui se suivent _(difficile d'interpoler la 650ème missing value du bloc sans les valeurs qui la précédent et la suivent)_.

In [ ]:
df_train = df_train.drop(index=range(4176, 5439)).reset_index(drop=True)

# Affichage de la heatmap des missing values
plt.figure(figsize=(14, 6))
sns.heatmap(df_train.isnull(), cbar=False, cmap="Blues")
plt.title("HeatMap des missing values (df_train)", fontsize=14)
plt.show()